### Notes
- 第一階段剪枝：Weight-Pruning
- 用於alarm, moaning, enghlish-help, misc-sounds classifications
- 剪枝比例：0.8-0.9, 建議0.85.

In [22]:
project_PATH = '/home/sail/sound_project/sound_ai_v2'
step_now = 'STEP/step_2'

In [23]:
use_model_in_step1 = "/home/sail/sound_project/sound_ai_v2/STEP/step_1/save_pt_model_s1/20240829_11/base4C_train_lr0.05_bs32_wd0.0005_20240829115502/uec_model_4Classes_hacc88.0_valacc_88.0_tracc_85.64453125_428th_epoch.pt"

In [24]:
import os;
import sys;
sys.path.append(os.path.abspath(f'{project_PATH}'))

import glob;
import math;
import numpy as np;
import random;
import time;
import torch
import torch.optim as optim;
import torch.nn as nn;

In [25]:
import common.utils as U;
import common.opts as opt;
# import th.resources.models as models;
# import th.resources.calculator as calc;
# import th.resources.train_generator as train_generator;
import th.resources.pruning_tools.weight_pruning as weight_pruner;

In [26]:
import argparse
import common.tlopts as tlopts
import th.resources.calculator as calc;
from datetime import datetime;
import pathlib

In [27]:
date_time = datetime.now().strftime("%Y%m%d_%H")  # %M
save_pt_model_path = f'{project_PATH}/{step_now}/save_pt_model_s2/{date_time}'
try:
    os.mkdir(save_pt_model_path)
    print(f"Folder '{save_pt_model_path}' created successfully.")
except FileExistsError:
    print(f"Folder '{save_pt_model_path}' already exists.")

Folder '/home/sail/sound_project/sound_ai_v2/STEP/step_2/save_pt_model_s2/20240829_14' already exists.


In [28]:
inp_len = 20150
sr = 20150
choose_class=[0,1,2,3,4,5,6]
PATH = f'{project_PATH}/STEP/data_v2.npz'

ch_n_class = 7
fcn_no_of_inputs = 7

In [29]:
data = np.load(PATH, allow_pickle=True) 
audio_max_value =  float((data['sounds_train']).max())*2 # the max value of the sound samples
audio_min_value =  float((data['sounds_train']).min())*2 # the max value of the sound samples
audio_max_value, audio_min_value

(3.0242209434509277, -3.1673121452331543)

In [30]:
order = [0,1,2,3,4,5,6]

def one_hot_encode(data, order):
    index_map = {number: index for index, number in enumerate(order)}
    one_hot_list = []

    for num in data:
        one_hot = [0] * len(order)
        if num in index_map:
            one_hot[index_map[num]] = 1
        one_hot_list.append(one_hot)

    return one_hot_list

In [31]:
#log file object
logObj = None;
def ChkAndCreateSingleDir(dir_path):
    if not pathlib.Path(dir_path).is_dir():
        os.mkdir(dir_path);
        print(f"'{dir_path}' folder is created.");

In [32]:
def genDataTimeStr():
    return datetime.today().strftime('%Y-%m-%d %H:%M:%S').replace('-',"").replace(' ',"").replace(':',"");

def getDateStr():
    return datetime.today().strftime('%Y-%m-%d %H').replace('-',"").replace(' ',"")#.replace(':',"");

In [33]:
class TLGenerator():
    #Generates data for Keras
    def __init__(self, samples, labels, options):
        random.seed(seed);
        #Initialization
        print(f"length of samples:{len(samples)}")
        self.data = [(samples[i], labels[i]) for i in range (0, len(samples))];
        self.opt = options;
        self.batch_size = options.batchSize;
        self.preprocess_funcs = self.preprocess_setup();
        self.mapdict = {0:1, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7,}
        # dict([('52',1),('56',2),('99',3)])

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.data) / self.batch_size));
        #return len(self.samples);

    def __getitem__(self, batchIndex):
        #Generate one batch of data
        batchX, batchY = self.generate_batch_select_fixed_class(batchIndex);
        batchX = np.expand_dims(batchX, axis=1);
        batchX = np.expand_dims(batchX, axis=3);
        return batchX, batchY

    def generate_batch(self, batchIndex):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                sound1, label1 = self.data[random.randint(0, len(self.data) - 1)]
                sound2, label2 = self.data[random.randint(0, len(self.data) - 1)]
                if label1 != label2:
                    break
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random())
            sound = U.mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            # print(f"sound length after U.mix is {len(sound)}")
            eye = np.eye(self.opt.nClasses)
            idx1 = self.mapdict[str(label1)]- 1
            idx2 = self.mapdict[str(label2)] - 1
            label = (eye[idx1] * r + eye[idx2] * (1 - r)).astype(np.float32)
            # label = (eye[label1] * r + eye[label2] * (1 - r)).astype(np.float32)

            #For stronger augmentation
            sound = U.random_gain(6)(sound).astype(np.float32)
            # print(f"sound length after U.random_gain is {len(sound)}")
            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        
        return sounds, labels;

    def generate_batch_select_fixed_class(self, batchIndex):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        #two variables recording alarm and moaning sounds count
        alarm_selected, help_eng_selected, help_ch_selected, help_ja_selected, help_tw_selected, help_hk_selected = 0, 0, 0, 0, 0, 0     
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                # print("enter while true")
                sound1, label1 = self.data[random.randint(0, len(self.data) - 1)]
                sound2, label2 = self.data[random.randint(0, len(self.data) - 1)]
                lbl1_int = np.int16(label1);
                lbl2_int = np.int16(label2);
                if label1 != label2:
                    # {'alarm': 0, 'en_help': 1, 'ch_help': 2, 'ja_help': 3, 'tw_help': 4, 'hk_help': 5, 'Environment': 6, 'other': 7}
                    if (lbl1_int == 0 and lbl2_int == 6) or (lbl1_int == 6 and lbl2_int ==0):
                        if (alarm_selected == help_ch_selected) and (alarm_selected == help_eng_selected):
                            alarm_selected += 1;
                            break;
                    if (lbl1_int == 1 and lbl2_int == 6) or (lbl1_int == 6 and lbl2_int == 1):
                        if (help_eng_selected < alarm_selected) and (help_eng_selected == help_ch_selected):
                            help_eng_selected += 1;
                            break;
                    if (lbl1_int == 2 and lbl2_int == 6) or (lbl1_int == 6 and lbl2_int == 2):
                        if (help_ch_selected < alarm_selected) and (help_ch_selected < help_eng_selected):
                            help_ch_selected += 1;
                            break;
                    if (lbl1_int == 3 and lbl2_int == 6) or (lbl1_int == 6 and lbl2_int == 3):
                        if (help_ja_selected < alarm_selected) and (help_ja_selected < help_ch_selected):
                            help_ja_selected += 1;
                            break;
                    if (lbl1_int == 4 and lbl2_int == 6) or (lbl1_int == 6 and lbl2_int == 4):
                        if (help_tw_selected < alarm_selected) and (help_tw_selected < help_ch_selected):
                            help_tw_selected += 1;
                            break;
                    if (lbl1_int == 5 and lbl2_int == 6) or (lbl1_int == 5 and lbl2_int == 4):
                        if (help_hk_selected < alarm_selected) and (help_hk_selected < help_ch_selected):
                            help_hk_selected += 1;
                            break;
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random());
            #######make wanted class mix ration above 0.5##########
            # iLbl1 = np.int16(label1);
            # iLbl2 = np.int16(label2);
            # r = 1.0;
            # p_ratio1 = 0.4
            # p_ratio2 = 0.6
            # while True:
            #     r = np.array(random.random());
            #     if r > p_ratio1 and iLbl1 != 99 :
            #         break;
            #     if r < p_ratio2 and iLbl2 != 99 :
            #         break;
            #######################End#######################
            # print(f"r:{r}, lbl1:{label1}, lbl2:{label2}")  
            sound = U.mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            eye = np.eye(self.opt.nClasses)
            idx1 = label1 # self.mapdict[str(label1)]- 1
            idx2 = label2 # self.mapdict[str(label2)] - 1
            label = (eye[idx1] * r + eye[idx2] * (1 - r)).astype(np.float32)
            

            #For stronger augmentation
            sound = U.random_gain(6)(sound).astype(np.float32)
            # print(f"sound length after U.random_gain is {len(sound)}")
            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        # print(f"batchIndex is {batchIndex}, total sounds is {len(sounds)}")
        # print(f"labels in generate_batch is:\n{labels}")
        # print(f"alarm_selected:{alarm_selected}, moaning_selected:{moaning_selected}");
        return sounds, labels;

    def preprocess_setup(self):
        funcs = []
        if self.opt.strongAugment:
            funcs += [U.random_scale(1.25)]

        funcs += [U.padding(self.opt.inputLength // 2),
                  U.random_crop(self.opt.inputLength),
                  U.regularization(audio_max_value, audio_min_value)]
        return funcs

    def preprocess(self, sound):
        for f in self.preprocess_funcs:
            sound = f(sound)

        return sound;

In [34]:
seed = 1123;
random.seed(seed);
np.random.seed(seed);
torch.manual_seed(seed);
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed);
torch.backends.cudnn.deterministic = True;
torch.backends.cudnn.benchmark = False;

In [35]:
class Customed_ACDNetV2(nn.Module):
    def __init__(self, input_length, n_class, sr, ch_conf=None):
        super(Customed_ACDNetV2, self).__init__()
        self.input_length = input_length
        self.ch_config = ch_conf

        stride1 = 2
        stride2 = 2
        channels = 8
        k_size = (3, 3)
        n_frames = (sr / 1000) * 10  # No of frames per 10ms

        sfeb_pool_size = int(n_frames / (stride1 * stride2))
        if self.ch_config is None:
            self.ch_config = [channels,
                              channels * 8,
                              channels * 4,
                              channels * 8, channels * 8,
                              channels * 16, channels * 16,
                              channels * 32, channels * 32,
                              channels * 64, channels * 64, n_class]

        ch_confing_10 = 8 * 32  # 8 * 64
        ch_n_class = n_class
        conv1, bn1 = self.make_layers(1, self.ch_config[0], (1, 9), (1, stride1))
        conv2, bn2 = self.make_layers(self.ch_config[0], self.ch_config[1], (1, 5), (1, stride2))
        conv3, bn3 = self.make_layers(1, self.ch_config[2], k_size, padding=1)
        conv4, bn4 = self.make_layers(self.ch_config[2], self.ch_config[3], k_size, padding=1)
        conv5, bn5 = self.make_layers(self.ch_config[3], self.ch_config[4], k_size, padding=1)
        conv6, bn6 = self.make_layers(self.ch_config[4], self.ch_config[5], k_size, padding=1)
        conv7, bn7 = self.make_layers(self.ch_config[5], self.ch_config[6], k_size, padding=1)
        conv8, bn8 = self.make_layers(self.ch_config[6], self.ch_config[7], k_size, padding=1)
        conv9, bn9 = self.make_layers(ch_confing_10, ch_n_class, (1, 1))
        fcn = nn.Linear(fcn_no_of_inputs, ch_n_class)
        nn.init.kaiming_normal_(fcn.weight, nonlinearity='sigmoid')  # Kaiming with sigmoid is equivalent to LeCun normal in Keras

        self.sfeb = nn.Sequential(
            # Start: Filter bank
            conv1, bn1, nn.ReLU(),
            conv2, bn2, nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, sfeb_pool_size))
        )

        tfeb_modules = []
        self.tfeb_width = int(((self.input_length / sr) * 1000) / 10)  # 10ms frames of audio length in seconds
        tfeb_pool_sizes = self.get_tfeb_pool_sizes(self.ch_config[1], self.tfeb_width)
        p_index = 0
        for i in [3, 4, 6, 8]:  # ,10
            tfeb_modules.extend([eval('conv{}'.format(i)), eval('bn{}'.format(i)), nn.ReLU()])

            if i != 3:
                tfeb_modules.extend([eval('conv{}'.format(i + 1)), eval('bn{}'.format(i + 1)), nn.ReLU()])

            h, w = tfeb_pool_sizes[p_index]
            if h > 1 or w > 1:
                if i == 8:
                    break
                else:
                    tfeb_modules.append(nn.MaxPool2d(kernel_size=(h, w)))
            p_index += 1

        tfeb_modules.append(nn.Dropout(0.2))
        # tfeb_modules.extend([conv9, bn9, nn.ReLU()])
        h, w = tfeb_pool_sizes[-1]
        if h > 1 or w > 1:
            tfeb_modules.append(nn.AvgPool2d(kernel_size=(5,7))) # h, w
        tfeb_modules.extend([nn.Flatten(), fcn])

        self.tfeb = nn.Sequential(*tfeb_modules)

        self.output = nn.Sequential(
            
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.sfeb(x)
        x = x.permute((0, 2, 1, 3))
        x = self.tfeb(x)
        y = self.output[0](x)
        return y

    def make_layers(self, in_channels, out_channels, kernel_size, stride=(1, 1), padding=0, bias=False):
        conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                          stride=stride, padding=padding, bias=bias)
        nn.init.kaiming_normal_(conv.weight, nonlinearity='relu')  # Kaiming with ReLU
        bn = nn.BatchNorm2d(out_channels)
        return conv, bn

    def get_tfeb_pool_sizes(self, con2_ch, width):
        h = self.get_tfeb_pool_size_component(con2_ch)
        w = self.get_tfeb_pool_size_component(width)
        pool_size = []
        for (h1, w1) in zip(h, w):
            pool_size.append((h1, w1))
        return pool_size

    def get_tfeb_pool_size_component(self, length):
        c = []
        index = 1
        while index <= 6:
            if length >= 2:
                if index == 6:
                    c.append(length)
                else:
                    c.append(2)
                    length = length // 2
            else:
                c.append(1)
            index += 1
        return c

def GetCustomedACDNetModel(input_len=inp_len, nclass=ch_n_class, sr=sr, channel_config=None):
    net = Customed_ACDNetV2(input_len, nclass, sr, ch_conf=channel_config);
    return net;

In [ ]:
models = torch.load(use_model_in_step2, map_location='cuda:0')['weight']
for i in models.keys():
    print(i, models[i].shape)

sfeb.0.weight torch.Size([8, 1, 1, 9])
sfeb.1.weight torch.Size([8])
sfeb.1.bias torch.Size([8])
sfeb.1.running_mean torch.Size([8])
sfeb.1.running_var torch.Size([8])
sfeb.1.num_batches_tracked torch.Size([])
sfeb.3.weight torch.Size([64, 8, 1, 5])
sfeb.4.weight torch.Size([64])
sfeb.4.bias torch.Size([64])
sfeb.4.running_mean torch.Size([64])
sfeb.4.running_var torch.Size([64])
sfeb.4.num_batches_tracked torch.Size([])
tfeb.0.weight torch.Size([32, 1, 3, 3])
tfeb.1.weight torch.Size([32])
tfeb.1.bias torch.Size([32])
tfeb.1.running_mean torch.Size([32])
tfeb.1.running_var torch.Size([32])
tfeb.1.num_batches_tracked torch.Size([])
tfeb.4.weight torch.Size([64, 32, 3, 3])
tfeb.5.weight torch.Size([64])
tfeb.5.bias torch.Size([64])
tfeb.5.running_mean torch.Size([64])
tfeb.5.running_var torch.Size([64])
tfeb.5.num_batches_tracked torch.Size([])
tfeb.7.weight torch.Size([64, 64, 3, 3])
tfeb.8.weight torch.Size([64])
tfeb.8.bias torch.Size([64])
tfeb.8.running_mean torch.Size([64])
tfeb.8

/tmp/ipykernel_1163137/312357234.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  models = torch.load(use_model_in_step2, map_location='cuda:0')['weight']


In [37]:
def getOpts():
    parser = argparse.ArgumentParser(description='Transfer Learning for ACDNet');
    parser.add_argument('--netType', default='TLACDNet',  required=False);
    parser.add_argument('--data', default='../datasets/processed/',  required=False);
    parser.add_argument('--dataset', required=False, default='uec_iot', choices=['10']);
    parser.add_argument('--BC', default=True, action='store_true', help='BC learning');
    parser.add_argument('--strongAugment', default=True,  action='store_true', help='Add scale and gain augmentation');
    #在ipynb中，不能使用parser.parse，要改用parser.parse_known_args()
    opt, unknown = parser.parse_known_args()
    #Leqarning settings
   
    opt.batchSize = 32;
    opt.LR = 0.05;
    opt.momentum = 0.09;
    opt.weightDecay = 5e-4;
    opt.schedule = [0.1,0.3,0.5,0.9];#default:[0.15, 0.30, 0.45, 0.60, 0.75];
    opt.warmup = 10;
    opt.nEpochs = 1000;  ############33

    #Basic Net Settings
    opt.nClasses = ch_n_class#50;
    opt.nFolds = 1;#5;
    opt.split = 1#[i for i in range(1, opt.nFolds + 1)];
    opt.sr = sr;
    opt.inputLength = inp_len;
    #Test data
    opt.nCrops = 2;
    return opt

- <font size=2 color='#FF6600'>For the accuracy and model generation capacity it is better to add more data to the training and validation datasets.</font>

In [38]:
def getTrainGen(opt=None, split=None):
    
    dataset = np.load(opt.Data_npz_path, allow_pickle=True);
    train_sounds = []
    train_labels = []

    train_sounds = dataset['sounds_train']
    train_labels = dataset['labels_train']

    trainGen = TLGenerator(train_sounds, train_labels, opt);
    return trainGen

In [39]:
class PruningTrainer:
    global logObj
    def __init__(self, opt):
        self.opt = opt;
        #Conditional compression settings
        # self.opt.LR = 0.01;
        # self.opt.momentum = 0.09;
        # self.opt.schedule = [0.15, 0.30, 0.45, 0.60, 0.75];
        # self.opt.warmup = 0;
        self.opt.prune_algo = 'l0norm';
        self.opt.prune_interval = 1;
        # self.opt.nEpochs = 1000;
        self.testX = None;
        self.testY = None;
        self.bestAcc = 0.0;
        self.bestAccEpoch = 0;
        self.trainGen = getTrainGen(opt)#train_generator.setup(self.opt, self.opt.split);
        # if torch.device == "cuda:0":
        #     self.device = '"cuda:0"'
        # elif torch.device == "mps":
        #     self.device = "mps"
        # else:
        #     self.device = "cpu"
        # 
        # self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu");

        if torch.cuda.is_available():
            self.opt.device = 'cuda:0'
        else:
            self.opt.device = 'cpu'
        print(f"!!! In PruningTrainer:: current used device:{self.opt.device}")

        self.start_time = time.time();

    def PruneAndTrain(self):
        self.trainGen;
        print(self.opt.device);
        loss_func = torch.nn.KLDivLoss(reduction='batchmean');

        #Load saved model dict net.load_state_dict(torch.load(base_model_path, map_location=self.opt.device)['weight'], strict=False)
        net = GetCustomedACDNetModel()#GetACDNetModel()
        base_model_path = use_model_in_step1

        net.load_state_dict(torch.load(base_model_path, map_location=self.opt.device)['weight'] ,strict=False);
        calc.summary(net, (1,1,self.opt.inputLength))
        net.eval();
        val_acc, val_loss = self.__validate(net, loss_func);
        print('Testing - Val: Loss {:.3f}  Acc(top1) {:.3f}%'.format(val_loss, val_acc));
        net.train();

        optimizer = optim.SGD(net.parameters(), lr=self.opt.LR, weight_decay=self.opt.weightDecay, momentum=self.opt.momentum, nesterov=True)

        weight_name = ["weight"]# if not self.opt.factorize else ["weightA", "weightB", "weightC"]
        layers_n = weight_pruner.layers_n(net, param_name=["weight"])[1];
        all_num = sum(layers_n.values());
        print("\t TOTAL PRUNABLE PARAMS: {}".format(all_num));
        print("\t PRUNE RATIO :{}".format(self.opt.prune_ratio));
        sparse_factor = int(all_num * (1-self.opt.prune_ratio));
        print("\t SPARSE FACTOR: {}".format(sparse_factor));
        model_size = (sparse_factor * 4)/1024**2;
        print("\t MODEL SIZE: {:.2f} MB".format(model_size));
        prune_algo = getattr(weight_pruner, self.opt.prune_algo);
        prune_func = lambda m: prune_algo(m, sparse_factor, param_name=weight_name);

        for epoch_idx in range(self.opt.nEpochs):
            epoch_start_time = time.time();
            optimizer.param_groups[0]['lr'] = self.__get_lr(epoch_idx+1);
            cur_lr = optimizer.param_groups[0]['lr'];
            running_loss = 0.0;
            running_acc = 0.0;
            n_batches = math.ceil(len(self.trainGen.data)/self.opt.batchSize);
            net.train();
            for batch_idx in range(n_batches):
                # with torch.no_grad():
                x,y = self.trainGen.__getitem__(batch_idx)
                x = torch.tensor(np.moveaxis(x, 3, 1)).to(self.opt.device);
                y = torch.tensor(y).to(self.opt.device);

                # zero the parameter gradients
                optimizer.zero_grad();

                # forward + backward + optimize
                # outputs = net(x);#in office and use cpu
                x = x.type(torch.FloatTensor)
                outputs = net(x);
                res_y = y.argmax(dim=1)
                res_y = res_y.type(torch.FloatTensor)
                running_acc += ((( outputs.data.argmax(dim=1) == res_y)*1).float().mean()).item();
                y = y.type(torch.FloatTensor)
                loss = loss_func(outputs.log(), y);

                loss.backward();
                optimizer.step();

                running_loss += loss.item();

                with torch.no_grad():
                    prune_func(net);

            prune_func(net)

            tr_acc = (running_acc / n_batches)*100;
            tr_loss = running_loss / n_batches;

            #Epoch wise validation Validation
            epoch_train_time = time.time() - epoch_start_time;
            net.eval();
            val_acc, val_loss = self.__validate(net, loss_func);
            #Save best model
            self.__save_model(val_acc, tr_acc, epoch_idx, net);

            self.__on_epoch_end(epoch_start_time, epoch_train_time, epoch_idx, cur_lr, tr_loss, tr_acc, val_loss, val_acc);

            running_loss = 0;
            running_acc = 0;
            net.train();

        total_time_taken = time.time() - self.start_time;
        print("Execution finished in: {}".format(U.to_hms(total_time_taken)));

    def load_test_data(self):
        data = np.load(self.opt.Data_npz_path, allow_pickle=True);
        print(f"device is :{self.opt.device}")
        print(f"len of Y:{len(data['labels_val'])}")
        dataX = data['sounds_val'].reshape(data['sounds_val'].shape[0],1,1,data['sounds_val'].shape[1]).astype(np.float32);
        self.testX = torch.tensor(dataX).to(self.opt.device);
        self.testY = torch.tensor(one_hot_encode(data['labels_val'], order)).type(torch.float32).to(self.opt.device);

    def __get_lr(self, epoch):
        divide_epoch = np.array([self.opt.nEpochs * i for i in self.opt.schedule]);
        decay = sum(epoch > divide_epoch);
        if epoch <= self.opt.warmup:
            decay = 1;
        return self.opt.LR * np.power(0.1, decay);

    def __validate(self, net, lossFunc):
        if self.testX is None:
            self.load_test_data()
        with torch.no_grad():
            y_pred = None;
            batch_size = (self.opt.batchSize//self.opt.nCrops)*self.opt.nCrops           
            
            for idx in range(math.ceil(len(self.testX)/batch_size)):
                x = self.testX[idx*batch_size : (idx+1)*batch_size];
                x = torch.tensor(x)
                x = x.type(torch.FloatTensor) # use apple mp2
                scores = net(x);
                y_pred = scores.data if y_pred is None else torch.cat((y_pred, scores.data));

            acc, loss = self.__compute_accuracy(y_pred, self.testY, lossFunc);
        return acc, loss;

    #Calculating average prediction (10 crops) and final accuracy
    def __compute_accuracy(self, y_pred, y_target, lossFunc):
        with torch.no_grad():
            #Reshape to shape theme like each sample comtains 10 samples, calculate mean and find theindices that has highest average value for each sample
            y_pred = (y_pred.reshape(y_pred.shape[0]//self.opt.nCrops, self.opt.nCrops, y_pred.shape[1])).mean(dim=1).argmax(dim=1);
            y_target = (y_target.reshape(y_target.shape[0]//self.opt.nCrops, self.opt.nCrops, y_target.shape[1])).mean(dim=1).argmax(dim=1);
            # print(f"y_pred:{type(y_pred)}, y_target:{type(y_target)}")
            y_target = y_target.cpu() #use apple m2, in office use cuda
            acc = (((y_pred==y_target)*1).float().mean()*100).item();
            # valLossFunc = torch.nn.KLDivLoss();
            loss = lossFunc(y_pred.float().log(), y_target.float()).item();
            # loss = 0.0;
        return acc, loss;

    def __on_epoch_end(self, epoch_start_time, train_time, epochIdx, lr, tr_loss, tr_acc, val_loss, val_acc):
        epoch_time = time.time() - epoch_start_time;
        val_time = epoch_time - train_time;
        total_time = time.time() - self.start_time;
        line = '{} Epoch: {}/{} | Time: {} (Train {}  Val {}) | Train: LR {}  Loss {:.2f}  Acc {:.2f}% | Val: Loss {:.2f}  Acc(top1) {:.2f}% | HA {:.2f}@{}\n'.format(
            U.to_hms(total_time), epochIdx+1, self.opt.nEpochs, U.to_hms(epoch_time), U.to_hms(train_time), U.to_hms(val_time),
            lr, tr_loss, tr_acc, val_loss, val_acc, self.bestAcc, self.bestAccEpoch);
        # print(line)
        sys.stdout.write(line);
        sys.stdout.flush();

    def __save_model(self, acc, train_acc, epochIdx, net):
        if acc > self.bestAcc and acc > self.opt.first_save_acc:
            self.bestAcc = acc;
            self.bestAccEpoch = epochIdx +1;
            self.__do_save_model(acc, train_acc, self.bestAccEpoch, net);
        else:
            if acc > self.opt.save_val_acc and train_acc > self.opt.save_train_acc: 
                self.__do_save_model(acc, train_acc, epochIdx, net);
            else:
                pass

    def __do_save_model(self, acc, tr_acc, epochIdx, net):
        save_model_name = self.opt.model_name.format(self.bestAcc, acc, tr_acc, epochIdx, genDataTimeStr());
        save_model_fullpath = self.opt.save_dir + save_model_name;
        print(f"save model to {save_model_fullpath}")
        torch.save({'weight':net.state_dict(), 'config':net.ch_config}, save_model_fullpath);
        logObj.write(f"save model:{self.opt.model_name}, bestAcc:{self.bestAcc}, valAcc:{acc}, trainAcc:{tr_acc}, record@{epochIdx}-epoch");
        logObj.write("\n");
        logObj.flush();

In [40]:
def main():
    global logObj;
    opt = getOpts()
    opt.Data_npz_path = PATH
    opt.sr = sr;
    opt.inputLength = inp_len;
    opt.trainer = None
    opt.prune_ratio = 0.9; # for faster we try the bigger pruning ratio first!!! 8/28 
    opt.first_save_acc = 65.0;
    opt.save_val_acc = 70.0;
    opt.save_train_acc = 85.0;
    trainStartTime = getDateStr();
    save_dir = f"{save_pt_model_path}/pruning_4C_time_{trainStartTime}_prunratio{opt.prune_ratio*100}/"
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    opt.save_dir = save_dir;
    opt.model_name = "uec_4Cmodel_first_stage_prun_haacc_{}_valacc{}_tracc{}_epoch_{}_{}.pt";
    print("Initializing PruneAndTrain Object.....")
    trainer = PruningTrainer(opt)#TLTrainer(opt)
    print("Start to pruning.....")
    logSaveDir = "./first_stage_pruning_logs/"
    ChkAndCreateSingleDir(logSaveDir);
    logName = "FirstPruningLog_{}.log".format(trainStartTime);
    logObj = open(os.path.join(logSaveDir,logName),'w');
    trainer.PruneAndTrain();
    logObj.flush();
    logObj.close();

In [41]:
main()

Initializing PruneAndTrain Object.....
length of samples:1018
!!! In PruningTrainer:: current used device:cuda:0
Start to pruning.....
cuda:0
plt
+----------------------------------------------------------------------------+
+                           Pytorch Model Summary                            +
------------------------------------------------------------------------------
   Layer (type)       Input Shape      Output Shape    Param #      FLOPS #
       Conv2d-1     (1, 1, 20150)     (8, 1, 10071)         72      725,112
  BatchNorm2d-2     (8, 1, 10071)     (8, 1, 10071)         16            0
         ReLu-3     (8, 1, 10071)     (8, 1, 10071)          0       80,568
       Conv2d-4     (8, 1, 10071)     (64, 1, 5034)      2,560   12,887,040
  BatchNorm2d-5     (64, 1, 5034)     (64, 1, 5034)        128            0
         ReLu-6     (64, 1, 5034)     (64, 1, 5034)          0      322,176
    MaxPool2d-7     (64, 1, 5034)      (64, 1, 100)          0      320,000
      Per

/tmp/ipykernel_1162272/194291979.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(base_model_path, map_location=self.opt.device)['weight']

0m08s Epoch: 1/1000 | Time: 0m08s (Train 0m08s  Val 0m00s) | Train: LR 0.005000000000000001  Loss 1.49  Acc 46.39% | Val: Loss nan  Acc(top1) 12.00% | HA 0.00@0
0m16s Epoch: 2/1000 | Time: 0m08s (Train 0m08s  Val 0m00s) | Train: LR 0.005000000000000001  Loss 1.26  Acc 48.05% | Val: Loss nan  Acc(top1) 16.00% | HA 0.00@0
0m25s Epoch: 3/1000 | Time: 0m08s (Train 0m08s  Val 0m00s) | Train: LR 0.005000000000000001  Loss 1.13  Acc 56.25% | Val: Loss nan  Acc(top1) 22.00% | HA 0.00@0
0m33s Epoch: 4/1000 | Time: 0m08s (Train 0m08s  Val 0m00s) | Train: LR 0.005000000000000001  Loss 1.12  Acc 53.32% | Val: Loss nan  Acc(top1) 30.00% | HA 0.00@0
0m42s Epoch: 5/1000 | Time: 0m08s (Train 0m08s  Val 0m00s) | Train: LR 0.005000000000000001  Loss 1.05  Acc 53.03% | Val: Loss nan  Acc(top1) 30.00% | HA 0.00@0
0m50s Epoch: 6/1000 | Time: 0m08s (Train 0m08s  Val 0m00s) | Train: LR 0.005000000000000001  Loss 1.05  Acc 50.29% | Val: Loss nan  Acc(top1) 28.00% | HA 0.00@0
0m58s Epoch: 7/1000 | Time: 0m08s 